# Notebook to visualize root of DTD



In [ ]:
%env CUDA_VISIBLE_DEVICES=""


from typing import Union, Callable

import torch 
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd

from lrp_relations import dtd


In [ ]:
torch.manual_seed(0)
mlp = dtd.NLayerMLP(4, 5, 10, 2)

def weight_scale(m: nn.Module) -> nn.Module:
    for p in m.parameters():
        # to compensate for negative biases, we scale the weight
        p.data[p.data > 0] = 1.4 * p.data[p.data > 0]
    if isinstance(m, dtd.LinearReLU):
        pass  # m.linear.bias.data = - m.linear.bias.data.abs() 
    return m

mlp.apply(weight_scale)

In [ ]:
x = torch.randn(10, 5)

logits = mlp(x)
logits

In [ ]:
roots = torch.cat([
    dtd.root_point_linear(x[:1], mlp.layer1, j=j)
    for j in range(mlp.layer1.out_features)
])

list(zip(roots, mlp.layer1(roots)))

In [ ]:

layer_root = torch.linalg.pinv(mlp.layer1.linear.weight) @ mlp.layer1.linear.bias 
mlp.layer1(layer_root)

In [ ]:
mlp.layer1(roots.mean(0, keepdim=True))